In [ ]:
# Import

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import numpy.matlib
import scipy.io
import scipy.stats as stats
import scipy.signal
from scipy.signal import find_peaks
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib Figure settings
mpl.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 26
plt.rcParams['axes.linewidth'] = 1
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [ ]:
# Load dataframe containing all units info and data

print('Loading data...')
with open('Data_5_Ephys_Opto_stGtACR1.pickle', 'rb') as f:
    clustInfoAll = pickle.load(f)
print('Loading complete. Ready to continue.')

In [ ]:
# ANALYSIS

print('Processing...')

#Set hardcoded parameters

# Set sounds and AM stim to analyze
AMlist = np.array([0.0, 1.0])  # AM modulation depth 0 (unmodulated noise) or 1 (10 Hz AM 100% depth)
sounds = np.unique(np.array(clustInfoAll.at[0, 'dB_stim']))  # Sound levels in dB SPL

# Some additional parameters for plotting
edges = np.arange(-1.0,1.5,.01)  # starting time, ending time, binSize - for raster and psth
time = edges[:-1] + np.diff(edges).mean()/2  # time vector 
# Create function 'smooth' for smoothing PSTHs
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

# Define time windows to use
SoundDelay = 0.250  # Sound onset delay after odor valve opening
BB_SoundDuration = 0.250  # Sound duration window to analyze for unmodulated broadband noise stim
BB_BLtime = (time>(-1)) & (time<0)  # Baseline window to analyze for unmodulated broadband noise stim z-scoring
AM_SoundDuration = 1.0  # Sound duration window to analyze for 10 Hz AM noise stim
AM_BLtime = (time>(-1)) & (time<0)  # Baseline window to analyze for 10 Hz AM noise stim z-scoring


# Run GLMs: LED_off and LED_ON

# BB stim (unmodulated broadband noise)
AM = 0.0
soundtime = (time>SoundDelay) & (time<(SoundDelay+BB_SoundDuration))
# create new dataframe columns to fill in GLM results
clustInfoAll['GLM_BB_LEDoff_const_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_dB_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Odor_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Interaction_coef'] = np.nan
clustInfoAll['GLM_BB_LEDoff_const_p'] = np.nan
clustInfoAll['GLM_BB_LEDoff_dB_p'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Odor_p'] = np.nan
clustInfoAll['GLM_BB_LEDoff_Interaction_p'] = np.nan
clustInfoAll['GLM_BB_LEDON_const_coef'] = np.nan
clustInfoAll['GLM_BB_LEDON_dB_coef'] = np.nan
clustInfoAll['GLM_BB_LEDON_Odor_coef'] = np.nan
clustInfoAll['GLM_BB_LEDON_Interaction_coef'] = np.nan
clustInfoAll['GLM_BB_LEDON_const_p'] = np.nan
clustInfoAll['GLM_BB_LEDON_dB_p'] = np.nan
clustInfoAll['GLM_BB_LEDON_Odor_p'] = np.nan
clustInfoAll['GLM_BB_LEDON_Interaction_p'] = np.nan
# LED off
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    indspikecountPSTH = indPSTH*0.01
    indspikesum = indspikecountPSTH[:, soundtime].sum(axis=1)
    LEDmask = ((indLED == 0) & (indAM == AM))
    indspikesum = indspikesum[LEDmask]
    inddB = inddB[LEDmask]
    indVial = indVial[LEDmask]
    if indspikesum.sum() == 0:
        continue
    inddB_new = np.zeros(len(inddB))
    indVial_new = np.zeros(len(indVial))
    inddB_new = inddB / np.max(inddB)
    for j,v in enumerate(np.unique(indVial)):
        indVial_new[indVial==v] = j+1
    indVial_new = indVial_new-1
    X = np.asarray([inddB_new, indVial_new, inddB_new*indVial_new])
    X = np.transpose(X)
    Xdf = pd.DataFrame(X, columns= ['dB', 'Odor', 'Interaction'])
    Xconst = sm.add_constant(Xdf)
    GLM_model = sm.GLM(indspikesum, Xconst, family=sm.families.Poisson())
    GLM_results = GLM_model.fit()
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_const_coef'] = GLM_results.params['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_const_p'] = GLM_results.pvalues['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_dB_coef'] = GLM_results.params['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_dB_p'] = GLM_results.pvalues['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Odor_coef'] = GLM_results.params['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Odor_p'] = GLM_results.pvalues['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Interaction_coef'] = GLM_results.params['Interaction']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDoff_Interaction_p'] = GLM_results.pvalues['Interaction']
# LED ON
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    indspikecountPSTH = indPSTH*0.01
    indspikesum = indspikecountPSTH[:, soundtime].sum(axis=1)
    LEDmask = ((indLED == 1) & (indAM == AM))
    indspikesum = indspikesum[LEDmask]
    inddB = inddB[LEDmask]
    indVial = indVial[LEDmask]
    if indspikesum.sum() == 0:
        continue
    inddB_new = np.zeros(len(inddB))
    indVial_new = np.zeros(len(indVial))
    inddB_new = inddB / np.max(inddB)
    for j,v in enumerate(np.unique(indVial)):
        indVial_new[indVial==v] = j+1
    indVial_new = indVial_new-1
    X = np.asarray([inddB_new, indVial_new, inddB_new*indVial_new])
    X = np.transpose(X)
    Xdf = pd.DataFrame(X, columns= ['dB', 'Odor', 'Interaction'])
    Xconst = sm.add_constant(Xdf)
    GLM_model = sm.GLM(indspikesum, Xconst, family=sm.families.Poisson())
    GLM_results = GLM_model.fit()
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_const_coef'] = GLM_results.params['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_const_p'] = GLM_results.pvalues['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_dB_coef'] = GLM_results.params['dB'] #/np.max(inddB)
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_dB_p'] = GLM_results.pvalues['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_Odor_coef'] = GLM_results.params['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_Odor_p'] = GLM_results.pvalues['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_Interaction_coef'] = GLM_results.params['Interaction'] #/np.max(inddB)
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_BB_LEDON_Interaction_p'] = GLM_results.pvalues['Interaction']

# AM stim (10 Hz AM noise)
AM = 1.0
soundtime = (time>SoundDelay) & (time<(SoundDelay+AM_SoundDuration))
# create new dataframe columns to fill in GLM results
clustInfoAll['GLM_AM_LEDoff_const_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_dB_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Odor_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Interaction_coef'] = np.nan
clustInfoAll['GLM_AM_LEDoff_const_p'] = np.nan
clustInfoAll['GLM_AM_LEDoff_dB_p'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Odor_p'] = np.nan
clustInfoAll['GLM_AM_LEDoff_Interaction_p'] = np.nan
clustInfoAll['GLM_AM_LEDON_const_coef'] = np.nan
clustInfoAll['GLM_AM_LEDON_dB_coef'] = np.nan
clustInfoAll['GLM_AM_LEDON_Odor_coef'] = np.nan
clustInfoAll['GLM_AM_LEDON_Interaction_coef'] = np.nan
clustInfoAll['GLM_AM_LEDON_const_p'] = np.nan
clustInfoAll['GLM_AM_LEDON_dB_p'] = np.nan
clustInfoAll['GLM_AM_LEDON_Odor_p'] = np.nan
clustInfoAll['GLM_AM_LEDON_Interaction_p'] = np.nan
# LED off
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    indspikecountPSTH = indPSTH*0.01
    indspikesum = indspikecountPSTH[:, soundtime].sum(axis=1)
    LEDmask = ((indLED == 0) & (indAM == AM))
    indspikesum = indspikesum[LEDmask]
    inddB = inddB[LEDmask]
    indVial = indVial[LEDmask]
    if indspikesum.sum() == 0:
        continue
    inddB_new = np.zeros(len(inddB))
    indVial_new = np.zeros(len(indVial))
    inddB_new = inddB / np.max(inddB)
    for j,v in enumerate(np.unique(indVial)):
        indVial_new[indVial==v] = j+1
    indVial_new = indVial_new-1
    X = np.asarray([inddB_new, indVial_new, inddB_new*indVial_new])
    X = np.transpose(X)
    Xdf = pd.DataFrame(X, columns= ['dB', 'Odor', 'Interaction'])
    Xconst = sm.add_constant(Xdf)
    GLM_model = sm.GLM(indspikesum, Xconst, family=sm.families.Poisson())
    GLM_results = GLM_model.fit()
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_const_coef'] = GLM_results.params['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_const_p'] = GLM_results.pvalues['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_dB_coef'] = GLM_results.params['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_dB_p'] = GLM_results.pvalues['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Odor_coef'] = GLM_results.params['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Odor_p'] = GLM_results.pvalues['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Interaction_coef'] = GLM_results.params['Interaction']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDoff_Interaction_p'] = GLM_results.pvalues['Interaction']
# LED ON
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    indspikecountPSTH = indPSTH*0.01
    indspikesum = indspikecountPSTH[:, soundtime].sum(axis=1)
    LEDmask = ((indLED == 1) & (indAM == AM))
    indspikesum = indspikesum[LEDmask]
    inddB = inddB[LEDmask]
    indVial = indVial[LEDmask]
    if indspikesum.sum() == 0:
        continue
    inddB_new = np.zeros(len(inddB))
    indVial_new = np.zeros(len(indVial))
    inddB_new = inddB / np.max(inddB)
    for j,v in enumerate(np.unique(indVial)):
        indVial_new[indVial==v] = j+1
    indVial_new = indVial_new-1
    X = np.asarray([inddB_new, indVial_new, inddB_new*indVial_new])
    X = np.transpose(X)
    Xdf = pd.DataFrame(X, columns= ['dB', 'Odor', 'Interaction'])
    Xconst = sm.add_constant(Xdf)
    GLM_model = sm.GLM(indspikesum, Xconst, family=sm.families.Poisson())
    GLM_results = GLM_model.fit()
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_const_coef'] = GLM_results.params['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_const_p'] = GLM_results.pvalues['const']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_dB_coef'] = GLM_results.params['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_dB_p'] = GLM_results.pvalues['dB']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_Odor_coef'] = GLM_results.params['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_Odor_p'] = GLM_results.pvalues['Odor']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_Interaction_coef'] = GLM_results.params['Interaction']
    clustInfoAll.at[clustInfoAll.index[ind], 'GLM_AM_LEDON_Interaction_p'] = GLM_results.pvalues['Interaction']


# ANALYSIS

# BB stim (unmodulated broadband noise)
# PSTHs, Responses, OMIs
AM = 0.0
BLtime =  BB_BLtime  # Baseline time window
soundtime = (time>SoundDelay) & (time<(SoundDelay+BB_SoundDuration))  # Response time window
#create dataframe columns
for dB in sounds:
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'].astype(object)
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'].astype(object)
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan
    clustInfoAll['OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
    clustInfoAll['OMI_'+str(dB)+'_'+str(AM)+'_LED'] = np.nan
    clustInfoAll['Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
    clustInfoAll['Delta_'+str(dB)+'_'+str(AM)+'_LED'] = np.nan
# individual unit analysis
for ind in clustInfoAll.index:
    #Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    for dB in sounds:
        NoOdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorLEDtrials = indPSTH[NoOdorLEDmask,:]
        NoOdorLEDresps = NoOdorLEDtrials[:,soundtime].mean(axis=1)
        NoOdorpsthLED = indPSTH[NoOdorLEDmask,:].mean(axis=0) 
        NoOdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorNoLEDtrials = indPSTH[NoOdorNoLEDmask,:]
        NoOdorNoLEDresps = NoOdorNoLEDtrials[:,soundtime].mean(axis=1)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorLEDtrials = indPSTH[OdorLEDmask,:]
        OdorLEDresps = OdorLEDtrials[:,soundtime].mean(axis=1)
        OdorpsthLED = indPSTH[OdorLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorNoLEDtrials = indPSTH[OdorNoLEDmask,:]
        OdorNoLEDresps = OdorNoLEDtrials[:,soundtime].mean(axis=1)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = [NoOdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [NoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = [OdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [OdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = NoOdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = NoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = OdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = OdorpsthNoLED[soundtime].mean()
        # OMI = Odor Modulation Index
        OMI_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean()) / (OdorpsthNoLED[soundtime].mean() + NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = OMI_LEDoff
        OMI_LEDON = (OdorpsthLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean()) / (OdorpsthLED[soundtime].mean() + NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'OMI_'+str(dB)+'_'+str(AM)+'_LED'] = OMI_LEDON
        # Delta = Odor - NoOdor
        Delta_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = Delta_LEDoff
        Delta_LEDON = (OdorpsthLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'Delta_'+str(dB)+'_'+str(AM)+'_LED'] = Delta_LEDON
# make arrays for each unit for avg sound responses (FR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_NoOdor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_Odor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_Odor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_Odor'].astype(object)
#get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayLED_NoOdor = []
    dBarrayNoLED_NoOdor = []
    dBarrayLED_Odor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_LED_NoOdor = psthLED_NoOdor[soundtime].mean()
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayLED_NoOdor = np.append(dBarrayLED_NoOdor, meanFR_LED_NoOdor)
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_LED_Odor = psthLED_Odor[soundtime].mean()
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayLED_Odor = np.append(dBarrayLED_Odor, meanFR_LED_Odor)
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = [dBarrayLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_LED_Odor'] = [dBarrayLED_Odor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Z scoring
# Get BLmean and BLstd for indiv neuron z-scoring
clustInfoAll['BL_FRmean'] = np.nan
clustInfoAll['BL_FRstd'] = np.nan
#create dataframe columns for Z-scored PSTHs
for dB in sounds:
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'].astype(object)
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'].astype(object)
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan   
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    BLalltrials = indPSTH[:,BLtime].mean(axis=0)
    BLmean = BLalltrials.mean()
    BLstd = BLalltrials.std()
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRmean'] = BLmean
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRstd'] = BLstd
    for dB in sounds:
        NoOdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorpsthLED = indPSTH[NoOdorLEDmask,:].mean(axis=0)
        NoOdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorpsthLED = indPSTH[OdorLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        zNoOdorpsthLED = (NoOdorpsthLED - (BLmean)) / BLstd            
        zNoOdorpsthNoLED = (NoOdorpsthNoLED - (BLmean)) / BLstd
        zOdorpsthLED = (OdorpsthLED - (BLmean)) / BLstd            
        zOdorpsthNoLED = (OdorpsthNoLED - (BLmean)) / BLstd
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = [zNoOdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [zNoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = [zOdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [zOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = zNoOdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = zNoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = zOdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = zOdorpsthNoLED[soundtime].mean()  
# make arrays for each unit for Z-Scored avg sound responses (zFR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_Odor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_Odor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_Odor'].astype(object)
# get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayLED_NoOdor = []
    dBarrayNoLED_NoOdor = []
    dBarrayLED_Odor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_LED_NoOdor = psthLED_NoOdor[soundtime].mean()
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayLED_NoOdor = np.append(dBarrayLED_NoOdor, meanFR_LED_NoOdor)
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_LED_Odor = psthLED_Odor[soundtime].mean()
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayLED_Odor = np.append(dBarrayLED_Odor, meanFR_LED_Odor)
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = [dBarrayLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_LED_Odor'] = [dBarrayLED_Odor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Get units that are overall positively or negatively modulated by odor (Overall OMI: responses to 55, 65, 75 dB)
clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED'] = np.nan  # Overall OMI
clustInfoAll['OdorEffect_v_'+str(AM)+'_LED'] = np.nan  # Overall OMI
clustInfoAll['LEDindexOdor_'+str(AM)] = np.nan  # LED Modulation Index
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    Soundmeans = indPSTH[:,soundtime].mean(axis=1)
    NoLED_NoOdor_mask = (indLED == 0) & (indVial == NoOdorVial) & (inddB >50) & (indAM == AM)
    NoLED_Odor_mask = (indLED == 0) & (indVial == OdorVial) & (inddB >50) & (indAM == AM)
    LED_NoOdor_mask = (indLED == 1) & (indVial == NoOdorVial) & (inddB >50) & (indAM == AM)
    LED_Odor_mask = (indLED == 1) & (indVial == OdorVial) & (inddB >50) & (indAM == AM)
    NoLED_NoOdor_Resp = Soundmeans[NoLED_NoOdor_mask]
    NoLED_Odor_Resp = Soundmeans[NoLED_Odor_mask]
    LED_NoOdor_Resp = Soundmeans[LED_NoOdor_mask]
    LED_Odor_Resp = Soundmeans[LED_Odor_mask]
    OdorEffectvalue = (NoLED_Odor_Resp.mean() - NoLED_NoOdor_Resp.mean()) / (NoLED_Odor_Resp.mean() + NoLED_NoOdor_Resp.mean())
    clustInfoAll.at[clustInfoAll.index[ind], 'OdorEffect_v_'+str(AM)+'_NoLED'] = OdorEffectvalue
    OdorEffectvalueLED = (LED_Odor_Resp.mean() - NoLED_NoOdor_Resp.mean()) / (LED_Odor_Resp.mean() + NoLED_NoOdor_Resp.mean()) 
    clustInfoAll.at[clustInfoAll.index[ind], 'OdorEffect_v_'+str(AM)+'_LED'] = OdorEffectvalueLED
    #LEDindex
    LEDindexOdor = (LED_Odor_Resp.mean() - NoLED_Odor_Resp.mean()) / (LED_Odor_Resp.mean() + NoLED_Odor_Resp.mean())
    clustInfoAll.at[clustInfoAll.index[ind], 'LEDindexOdor_'+str(AM)] = LEDindexOdor

# AM stim (10 Hz AM noise)
# PSTHs, Responses, OMIs
AM = 1.0
BLtime =  AM_BLtime  # Baseline time window
soundtime = (time>SoundDelay) & (time<(SoundDelay+AM_SoundDuration))  # Response time window
#create dataframe columns
for dB in sounds:
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'].astype(object)
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan
    clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = clustInfoAll['PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'].astype(object)
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['Resp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan
    clustInfoAll['OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
    clustInfoAll['OMI_'+str(dB)+'_'+str(AM)+'_LED'] = np.nan
    clustInfoAll['Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = np.nan
    clustInfoAll['Delta_'+str(dB)+'_'+str(AM)+'_LED'] = np.nan
# individual unit analysis
for ind in clustInfoAll.index:
    #Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    for dB in sounds:
        NoOdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorLEDtrials = indPSTH[NoOdorLEDmask,:]
        NoOdorLEDresps = NoOdorLEDtrials[:,soundtime].mean(axis=1)
        NoOdorpsthLED = indPSTH[NoOdorLEDmask,:].mean(axis=0) 
        NoOdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorNoLEDtrials = indPSTH[NoOdorNoLEDmask,:]
        NoOdorNoLEDresps = NoOdorNoLEDtrials[:,soundtime].mean(axis=1)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorLEDtrials = indPSTH[OdorLEDmask,:]
        OdorLEDresps = OdorLEDtrials[:,soundtime].mean(axis=1)
        OdorpsthLED = indPSTH[OdorLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorNoLEDtrials = indPSTH[OdorNoLEDmask,:]
        OdorNoLEDresps = OdorNoLEDtrials[:,soundtime].mean(axis=1)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = [NoOdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [NoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = [OdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [OdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = NoOdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = NoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = OdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'Resp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = OdorpsthNoLED[soundtime].mean()
        # OMI = Odor Modulation Index
        OMI_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean()) / (OdorpsthNoLED[soundtime].mean() + NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'OMI_'+str(dB)+'_'+str(AM)+'_NoLED'] = OMI_LEDoff
        OMI_LEDON = (OdorpsthLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean()) / (OdorpsthLED[soundtime].mean() + NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'OMI_'+str(dB)+'_'+str(AM)+'_LED'] = OMI_LEDON
        # Delta = Odor - NoOdor
        Delta_LEDoff = (OdorpsthNoLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'Delta_'+str(dB)+'_'+str(AM)+'_NoLED'] = Delta_LEDoff
        Delta_LEDON = (OdorpsthLED[soundtime].mean() - NoOdorpsthNoLED[soundtime].mean())
        clustInfoAll.at[clustInfoAll.index[ind], 'Delta_'+str(dB)+'_'+str(AM)+'_LED'] = Delta_LEDON
# make arrays for each unit for avg sound responses (FR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_NoOdor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_Odor'] = np.nan
clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_Odor'] = clustInfoAll['FR_dB_Plot_'+str(AM)+'_LED_Odor'].astype(object)
#get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayLED_NoOdor = []
    dBarrayNoLED_NoOdor = []
    dBarrayLED_Odor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_LED_NoOdor = psthLED_NoOdor[soundtime].mean()
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayLED_NoOdor = np.append(dBarrayLED_NoOdor, meanFR_LED_NoOdor)
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_LED_Odor = psthLED_Odor[soundtime].mean()
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayLED_Odor = np.append(dBarrayLED_Odor, meanFR_LED_Odor)
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = [dBarrayLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_LED_Odor'] = [dBarrayLED_Odor]
    clustInfoAll.at[clustInfoAll.index[ind], 'FR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Z scoring
# Get BLmean and BLstd for indiv neuron z-scoring
clustInfoAll['BL_FRmean'] = np.nan
clustInfoAll['BL_FRstd'] = np.nan
#create dataframe columns for Z-scored PSTHs
for dB in sounds:
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'].astype(object)
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'].astype(object)           
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'].astype(object)
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan
    clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = clustInfoAll['zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'].astype(object)
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = np.nan
    clustInfoAll['zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = np.nan   
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    BLalltrials = indPSTH[:,BLtime].mean(axis=0)
    BLmean = BLalltrials.mean()
    BLstd = BLalltrials.std()
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRmean'] = BLmean
    clustInfoAll.at[clustInfoAll.index[ind], 'BL_FRstd'] = BLstd
    for dB in sounds:
        NoOdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorpsthLED = indPSTH[NoOdorLEDmask,:].mean(axis=0)
        NoOdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == NoOdorVial)
        NoOdorpsthNoLED = indPSTH[NoOdorNoLEDmask,:].mean(axis=0)
        OdorLEDmask = (indLED == 1) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorpsthLED = indPSTH[OdorLEDmask,:].mean(axis=0)
        OdorNoLEDmask = (indLED == 0) & (inddB == dB) & (indAM == AM) & (indVial == OdorVial)
        OdorpsthNoLED = indPSTH[OdorNoLEDmask,:].mean(axis=0)
        zNoOdorpsthLED = (NoOdorpsthLED - (BLmean)) / BLstd            
        zNoOdorpsthNoLED = (NoOdorpsthNoLED - (BLmean)) / BLstd
        zOdorpsthLED = (OdorpsthLED - (BLmean)) / BLstd            
        zOdorpsthNoLED = (OdorpsthNoLED - (BLmean)) / BLstd
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = [zNoOdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = [zNoOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = [zOdorpsthLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = [zOdorpsthNoLED]
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor'] = zNoOdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor'] = zNoOdorpsthNoLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor'] = zOdorpsthLED[soundtime].mean()
        clustInfoAll.at[clustInfoAll.index[ind], 'zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor'] = zOdorpsthNoLED[soundtime].mean()  
# make arrays for each unit for Z-Scored avg sound responses (zFR_dB_Plot), for Odor and NoOdor, and add to dataframe
# create dataframe columns
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'].astype(object)
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_Odor'] = np.nan
clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_Odor'] = clustInfoAll['zFR_dB_Plot_'+str(AM)+'_LED_Odor'].astype(object)
# get individual PSTHs
for ind in clustInfoAll.index:
    dBarrayLED_NoOdor = []
    dBarrayNoLED_NoOdor = []
    dBarrayLED_Odor = []
    dBarrayNoLED_Odor = []       
    for dB in sounds:
        psthLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
        psthNoLED_NoOdor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
        meanFR_LED_NoOdor = psthLED_NoOdor[soundtime].mean()
        meanFR_NoLED_NoOdor = psthNoLED_NoOdor[soundtime].mean()
        dBarrayLED_NoOdor = np.append(dBarrayLED_NoOdor, meanFR_LED_NoOdor)
        dBarrayNoLED_NoOdor = np.append(dBarrayNoLED_NoOdor, meanFR_NoLED_NoOdor)
        psthLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
        psthNoLED_Odor = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'zPSTH_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
        meanFR_LED_Odor = psthLED_Odor[soundtime].mean()
        meanFR_NoLED_Odor = psthNoLED_Odor[soundtime].mean()
        dBarrayLED_Odor = np.append(dBarrayLED_Odor, meanFR_LED_Odor)
        dBarrayNoLED_Odor = np.append(dBarrayNoLED_Odor, meanFR_NoLED_Odor)
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_LED_NoOdor'] = [dBarrayLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_NoOdor'] = [dBarrayNoLED_NoOdor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_LED_Odor'] = [dBarrayLED_Odor]
    clustInfoAll.at[clustInfoAll.index[ind], 'zFR_dB_Plot_'+str(AM)+'_NoLED_Odor'] = [dBarrayNoLED_Odor]
# Get units that are overall positively or negatively modulated by odor (Overall OMI: responses to 55, 65, 75 dB)
clustInfoAll['OdorEffect_v_'+str(AM)+'_NoLED'] = np.nan  # Overall OMI
clustInfoAll['OdorEffect_v_'+str(AM)+'_LED'] = np.nan  # Overall OMI
clustInfoAll['LEDindexOdor_'+str(AM)] = np.nan  # LED Modulation Index
for ind in clustInfoAll.index:
    # Get individual PSTHs and Stimuli
    indPSTH = np.squeeze(np.array(clustInfoAll.at[clustInfoAll.index[ind], 'PSTH']))
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    inddB = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'dB_stim'])
    indAM = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'AM_stim'])
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    Soundmeans = indPSTH[:,soundtime].mean(axis=1)
    NoLED_NoOdor_mask = (indLED == 0) & (indVial == NoOdorVial) & (inddB >50) & (indAM == AM)
    NoLED_Odor_mask = (indLED == 0) & (indVial == OdorVial) & (inddB >50) & (indAM == AM)
    LED_NoOdor_mask = (indLED == 1) & (indVial == NoOdorVial) & (inddB >50) & (indAM == AM)
    LED_Odor_mask = (indLED == 1) & (indVial == OdorVial) & (inddB >50) & (indAM == AM)
    NoLED_NoOdor_Resp = Soundmeans[NoLED_NoOdor_mask]
    NoLED_Odor_Resp = Soundmeans[NoLED_Odor_mask]
    LED_NoOdor_Resp = Soundmeans[LED_NoOdor_mask]
    LED_Odor_Resp = Soundmeans[LED_Odor_mask]
    OdorEffectvalue = (NoLED_Odor_Resp.mean() - NoLED_NoOdor_Resp.mean()) / (NoLED_Odor_Resp.mean() + NoLED_NoOdor_Resp.mean())
    clustInfoAll.at[clustInfoAll.index[ind], 'OdorEffect_v_'+str(AM)+'_NoLED'] = OdorEffectvalue
    OdorEffectvalueLED = (LED_Odor_Resp.mean() - NoLED_NoOdor_Resp.mean()) / (LED_Odor_Resp.mean() + NoLED_NoOdor_Resp.mean()) 
    clustInfoAll.at[clustInfoAll.index[ind], 'OdorEffect_v_'+str(AM)+'_LED'] = OdorEffectvalueLED
    #LEDindex
    LEDindexOdor = (LED_Odor_Resp.mean() - NoLED_Odor_Resp.mean()) / (LED_Odor_Resp.mean() + NoLED_Odor_Resp.mean())
    clustInfoAll.at[clustInfoAll.index[ind], 'LEDindexOdor_'+str(AM)] = LEDindexOdor


# Create function 'bar_plot' for graphing bar plots with multiple groups
# adapted from https://stackoverflow.com/questions/14270391/how-to-plot-multiple-bars-grouped
def bar_plot(ax, data, errdata, colors=None, total_width=0.8, single_width=1, legend=True, legend_loc='best'):
    """Draws a bar plot with multiple bars per data point.
    Parameters
    ----------
    ax : matplotlib.pyplot.axis
        The axis we want to draw our plot on.
    data: dictionary
        A dictionary containing the data we want to plot. Keys are the names of the
        data, the items is a list of the values.
        Example:
        data = {
            "x":[1,2,3],
            "y":[1,2,3],
            "z":[1,2,3],
        }
    errdata: dictionary containing the errorbar data, with keys matching the corresponding data table
    colors : array-like, optional
        A list of colors which are used for the bars. If None, the colors
        will be the standard matplotlib color cyle. (default: None)
    total_width : float, optional, default: 0.8
        The width of a bar group. 0.8 means that 80% of the x-axis is covered
        by bars and 20% will be spaces between the bars.
    single_width: float, optional, default: 1
        The relative width of a single bar within a group. 1 means the bars
        will touch eachother within a group, values less than 1 will make
        these bars thinner.
    legend: bool, optional, default: True
        If this is set to true, a legend will be added to the axis.
    """
    # Check if colors where provided, otherwhise use the default color cycle
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    # Number of bars per group
    n_bars = len(data)
    # The width of a single bar
    bar_width = total_width / n_bars
    # List containing handles for the drawn bars, used for the legend
    bars = []
    # Iterate over all data
    for i, (name, values) in enumerate(data.items()):
        # The offset in x direction of that bar
        x_offset = (i - n_bars / 2) * bar_width + bar_width / 2
        # Draw a bar for every value of that type
        for x, y in enumerate(values):
            bar = ax.bar(x + x_offset, y, width=bar_width * single_width, color=colors[i % len(colors)], 
                         edgecolor=['black', 'black'], yerr=errdata[str(name)][x], capsize=10, fill=True)
        # Add a handle to the last drawn bar, which we'll need for the legend
        bars.append(bar[0])
    # Draw legend if we need
    if legend:
        ax.legend(bars, data.keys(), loc=legend_loc, prop={'size': 20})


print('Analysis complete. Ready to continue.')

In [ ]:
# FIGURE 5 A-D

fig, axs = plt.subplots(nrows=4,ncols=2,figsize=(12, 24), sharex=False)
fig.subplots_adjust(hspace=.4,wspace=0.3)
# BB stim (unmodulated broadband noise)
AM = 0.0
clustInfoMask = clustInfoAll.loc[((clustInfoAll['GLM_BB_LEDoff_dB_p']<0.05) &
                             (clustInfoAll['GLM_BB_LEDoff_dB_coef']>0) &
                             (clustInfoAll['Region']!='TeA') &     
                            (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_BB_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_BB_LEDoff_Interaction_p']<0.05)))]
total_n = len(clustInfoMask.index)
mouse_N = len(clustInfoMask.mouse.unique())
print(str(mouse_N)+ ' mice')
print(clustInfoMask.mouse.unique())
print(str(total_n)+ ' total units')
clustInfoENH = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']>=0))]
clustInfoSUPP = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']<0))]
Enh_n = len(clustInfoENH.index)
print(str(Enh_n)+' Enh. units')
Supp_n = len(clustInfoSUPP.index)
print(str(Supp_n)+' Supp. units')
#################################################################################################################
#### PLOT 1 : OMI SCATTER PLOT - 5A LEFT
plt.subplot(4,2,1)
OdoreffectEnh = clustInfoENH['OdorEffect_v_'+str(AM)+'_NoLED']
OdoreffectLEDEnh = clustInfoENH['OdorEffect_v_'+str(AM)+'_LED']
x = OdoreffectEnh
y = OdoreffectLEDEnh
OdoreffectSupp = clustInfoSUPP['OdorEffect_v_'+str(AM)+'_NoLED']
OdoreffectLEDSupp = clustInfoSUPP['OdorEffect_v_'+str(AM)+'_LED']
x2 = OdoreffectSupp
y2 = OdoreffectLEDSupp
print('SCATTER PLOT STATS:')
t, p = stats.shapiro(x)
print('Enhanced off shapiro p ='+str(p))
t, pp = stats.shapiro(y)
print('Enhanced ON shapiro p ='+str(pp))
if p<0.05 or pp<0.05:
    t, ep = stats.wilcoxon(x, y)
    print('Enhanced Wilcoxon Z= '+str(t))
    print('Enhanced Wilcoxon p= '+str(ep))
else:
    t, ep = stats.ttest_re;l(x, y)
    print('Enhanced Paired ttest t= '+str(t))
    print('Enhanced Paired ttest p= '+str(ep))
t, p = stats.shapiro(x2)
print('Suppressed off shapiro p ='+str(p))
t, pp = stats.shapiro(y2)
print('Suppressed ON shapiro p ='+str(pp))
if p<0.05 or pp<0.05:
    t, sp = stats.wilcoxon(x2, y2)
    print('Suppressed Wilcoxon Z= '+str(t))
    print('Suppressed Wilcoxon p= '+str(sp))
else:
    t, sp = stats.ttest_rel(x2, y2)
    print('Suppressed Paired ttest t= '+str(t))
    print('Suppressed Paired ttest p= '+str(sp))
plt.scatter(x, y, color='mediumvioletred', s=20)
plt.scatter(x2, y2, color='violet', s=20)
plt.legend(['Enhanced, p<0.01', 'Suppressed, n.s.'], loc='upper left', prop={'size': 20})
xmin = -0.5
xmax = 0.5
ymin = -0.5
ymax = 0.5
plt.xlabel('OMI (LED off)', color='magenta')
plt.ylabel('OMI (LED ON)', color='deepskyblue')
plt.xlim([xmin, xmax])
plt.xticks([xmin, 0, xmax])
plt.ylim([ymin, ymax])
plt.yticks([ymin, 0, ymax])
plt.axhline(0, color='black', ls='-')
plt.axvline(0, color='black', ls='-')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
unity = np.linspace(np.min([xmin, ymin]), np.max([xmax, ymax]))
plt.plot(unity, unity, color='black', ls='--', label='_nolegend_')
# LED Modulation Index save pickle for Fig 6 comparison
BB_LEDeffectEnh_Expt = clustInfoENH['LEDindexOdor_'+str(AM)]
with open('Data_6Eleft_LEDindex_stGtACR1.pickle', 'wb') as f:
    pickle.dump(BB_LEDeffectEnh_Expt, f)
#################################################################################################################
#### PLOT 2 : OMI BAR PLOTS - 5A RIGHT
plt.subplot(4,2,2)
data = {
        "LED off": [x.mean(), x2.mean()],
        "LED ON": [y.mean(), y2.mean()],
    }
errdata = {
        "LED off": [x.std()/np.sqrt(len(x)), x2.std()/np.sqrt(len(x2))],
        "LED ON": [y.std()/np.sqrt(len(y)), y2.std()/np.sqrt(len(y2))],
    }
bar_plot(axs[0,1], data, errdata, colors=['magenta', 'deepskyblue'], total_width=.8, single_width=.9, legend_loc='upper right')
plt.ylabel('OMI')
plt.xticks(ticks=[0, 1], labels=['Enhanced', 'Suppressed'])
plt.ylim([-0.2, 0.2])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 3 : ENHANCED DB LINE PLOT - 5B LEFT
plt.subplot(4,2,3)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
mean_dBarray_LED_NoOdor = []
mean_dBarray_LED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
sem_dBarray_LED_NoOdor = []
sem_dBarray_LED_Odor = []
print('ENHANCED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    LED_NoOdor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
    LED_Odor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoLED_NoOdor shapiro p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': NoLED_Odor shapiro. p= '+str(pp))
    t, ppp = stats.shapiro(LED_Odor)
    print(str(dB)+': LED_Odor shapiro p= '+str(ppp))
    if p<0.05 or pp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Wilcoxon W= '+str(t))
        print(str(dB)+' NoLED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Paired ttest t= '+str(t))
        print(str(dB)+' NoLED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    if p<0.05 or ppp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Wilcoxon W= '+str(t))
        print(str(dB)+' LED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Paired ttest t= '+str(t))
        print(str(dB)+' LED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    mean_dBarray_LED_NoOdor = np.append(mean_dBarray_LED_NoOdor, LED_NoOdor.mean())
    mean_dBarray_LED_Odor = np.append(mean_dBarray_LED_Odor, LED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
    sem_dBarray_LED_NoOdor = np.append(sem_dBarray_LED_NoOdor, LED_NoOdor.std()/np.sqrt(len(LED_NoOdor)))
    sem_dBarray_LED_Odor = np.append(sem_dBarray_LED_Odor, LED_Odor.std()/np.sqrt(len(LED_Odor)))
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_NoOdor[2:], sem_dBarray_NoLED_NoOdor[2:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_Odor[2:], sem_dBarray_NoLED_Odor[2:], capsize=5, marker='o', color='mediumvioletred')
plt.errorbar(sounds[2:]-50, mean_dBarray_LED_Odor[2:], sem_dBarray_LED_Odor[2:], capsize=5, marker='o', color='deepskyblue')
plt.legend(['Sound (control)', 'Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper left', prop={'size': 20})
plt.ylim([0, 40])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(np.asarray(sounds[2:]-50))
plt.ylabel('Avg. Firing Rate Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Enhanced')
#################################################################################################################
#### PLOT 4 : SUPPRESSED DB LINE PLOT - 5B RIGHT
plt.subplot(4,2,4)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
mean_dBarray_LED_NoOdor = []
mean_dBarray_LED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
sem_dBarray_LED_NoOdor = []
sem_dBarray_LED_Odor = []
print('SUPPRESSED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    LED_NoOdor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
    LED_Odor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoLED_NoOdor shapiro p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': NoLED_Odor shapiro. p= '+str(pp))
    t, ppp = stats.shapiro(LED_Odor)
    print(str(dB)+': LED_Odor shapiro p= '+str(ppp))
    if p<0.05 or pp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Wilcoxon W= '+str(t))
        print(str(dB)+' NoLED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Paired ttest t= '+str(t))
        print(str(dB)+' NoLED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    if p<0.05 or ppp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Wilcoxon W= '+str(t))
        print(str(dB)+' LED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Paired ttest t= '+str(t))
        print(str(dB)+' LED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    mean_dBarray_LED_NoOdor = np.append(mean_dBarray_LED_NoOdor, LED_NoOdor.mean())
    mean_dBarray_LED_Odor = np.append(mean_dBarray_LED_Odor, LED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
    sem_dBarray_LED_NoOdor = np.append(sem_dBarray_LED_NoOdor, LED_NoOdor.std()/np.sqrt(len(LED_NoOdor)))
    sem_dBarray_LED_Odor = np.append(sem_dBarray_LED_Odor, LED_Odor.std()/np.sqrt(len(LED_Odor)))
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_NoOdor[2:], sem_dBarray_NoLED_NoOdor[2:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_Odor[2:], sem_dBarray_NoLED_Odor[2:], capsize=5, marker='o', color='violet')
plt.errorbar(sounds[2:]-50, mean_dBarray_LED_Odor[2:], sem_dBarray_LED_Odor[2:], capsize=5, marker='o', color='lightskyblue')
plt.legend(['Sound (control)', 'Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper left', prop={'size': 20})
plt.ylim([0, 40])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(np.asarray(sounds[2:]-50))
# plt.ylabel('Avg. Firing Rate Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Suppressed')
#################################################################################################################
#### PLOT 5 : ENHANCED STATS COMPARISON BAR - 5C LEFT
plt.subplot(4,2,5)
mean_OMIarray_NoLED = []
mean_OMIarray_LED = []
sem_OMIarray_NoLED = []
sem_OMIarray_LED = []
K_data_off = {}
K_data_ON = {}
print('ENHANCED STATS OMI BAR:')
for i, dB in enumerate(np.asarray(sounds[2:])):
    NoLED = np.array((clustInfoENH['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    LED = np.array((clustInfoENH['OMI_'+str(dB)+'_'+str(AM)+'_LED']))
    K_data_off[str(dB)] = NoLED[~np.isnan(NoLED)]
    K_data_ON[str(dB)] = LED[~np.isnan(LED)]
    t, p = stats.shapiro(NoLED)
    print(str(dB)+': LED off shap. p= '+str(p))
    t, pp = stats.shapiro(LED)
    print(str(dB)+': LED ON shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED, LED)
        print(str(dB)+' Wilcoxon W= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_rel(NoLED, LED)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, NoLED.mean())
    mean_OMIarray_LED = np.append(mean_OMIarray_LED, LED.mean())
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, NoLED.std()/np.sqrt(len(NoLED)))
    sem_OMIarray_LED = np.append(sem_OMIarray_LED, LED.std()/np.sqrt(len(LED)))
data = {
        "LED off": mean_OMIarray_NoLED,
        "LED ON": mean_OMIarray_LED,
    }
errdata = {
        "LED off": sem_OMIarray_NoLED,
        "LED ON": sem_OMIarray_LED,
    }
bar_plot(axs[2,0], data, errdata, colors=['mediumvioletred', 'deepskyblue'], total_width=.8, single_width=.9, legend_loc='lower left')
plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['-5', '5', '15', '25'])
plt.ylim([-0.2, 0.2])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 6 : SUPPRESSED STATS COMPARISON BAR - 5C RIGHT
plt.subplot(4,2,6)
mean_OMIarray_NoLED = []
mean_OMIarray_LED = []
sem_OMIarray_NoLED = []
sem_OMIarray_LED = []
K_data_off = {}
K_data_ON = {}
print('SUPPRESSED STATS OMI BAR:')
for i, dB in enumerate(np.asarray(sounds[2:])):
    NoLED = np.array((clustInfoSUPP['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    LED = np.array((clustInfoSUPP['OMI_'+str(dB)+'_'+str(AM)+'_LED']))
    K_data_off[str(dB)] = NoLED[~np.isnan(NoLED)]
    K_data_ON[str(dB)] = LED[~np.isnan(LED)]
    t, p = stats.shapiro(NoLED)
    print(str(dB)+': LED off shap. p= '+str(p))
    t, pp = stats.shapiro(LED)
    print(str(dB)+': LED ON shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED, LED)
        print(str(dB)+' Wilcoxon W= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_rel(NoLED, LED)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, NoLED.mean())
    mean_OMIarray_LED = np.append(mean_OMIarray_LED, LED.mean())
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, NoLED.std()/np.sqrt(len(NoLED)))
    sem_OMIarray_LED = np.append(sem_OMIarray_LED, LED.std()/np.sqrt(len(LED)))
data = {
        "LED off": mean_OMIarray_NoLED,
        "LED ON": mean_OMIarray_LED,
    }
errdata = {
        "LED off": sem_OMIarray_NoLED,
        "LED ON": sem_OMIarray_LED,
    }
bar_plot(axs[2,1], data, errdata, colors=['violet', 'lightskyblue'], total_width=.8, single_width=.9, legend_loc='upper right')
# plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['-5', '5', '15', '25'])
plt.ylim([-0.2, 0.2])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 7 : ENHANCED PSTH - 5D LEFT
plt.subplot(4,2,7)
dB_sel = 75
clustInfoENH = clustInfoMask.loc[(clustInfoMask['OMI_'+str(dB_sel)+'_'+str(AM)+'_NoLED']>0)]
n = len(clustInfoENH.index)
PSTHall_LED_NoOdor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_NoOdor'])))
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_LED_Odor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_Odor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanLED_NoOdor = PSTHall_LED_NoOdor.mean(axis=0)
semLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanLED_Odor = PSTHall_LED_Odor.mean(axis=0)
semLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='mediumvioletred')
plt.plot(time, smooth(meanLED_Odor, 3), color='deepskyblue')
plt.legend(['Sound (control)','Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper right', prop={'size': 20})
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate Z-Scored')
plt.ylim(-5, 60)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='-')
plt.axvline(x=0, color='deepskyblue', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 8 : SUPPRESSED PSTH - 5D RIGHT
plt.subplot(4,2,8)
dB_sel = 75
clustInfoSUPP = clustInfoMask.loc[(clustInfoMask['OMI_'+str(dB_sel)+'_'+str(AM)+'_NoLED']<0)]
n = len(clustInfoSUPP.index)
PSTHall_LED_NoOdor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_NoOdor'])))
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_LED_Odor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_Odor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanLED_NoOdor = PSTHall_LED_NoOdor.mean(axis=0)
semLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanLED_Odor = PSTHall_LED_Odor.mean(axis=0)
semLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='violet')
plt.plot(time, smooth(meanLED_Odor, 3), color='lightskyblue')
plt.legend(['Sound (control)','Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper right', prop={'size': 20})
plt.xlabel('Time (s)')
# plt.ylabel('Avg. Firing Rate Z-Scored')
plt.ylim(-5, 60)
plt.xlim(-0.25, 1.5)
#plt.yticks([0, 5, 10, 15, 20, 25, 30])
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='-')
plt.axvline(x=0, color='deepskyblue', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')

# Save Fig
filename = 'Fig 5ABCD'+'.pdf'
plt.savefig(filename, bbox_inches='tight', dpi=300, transparent=False)

plt.show()

In [ ]:
# FIGURE 5 E-H

fig, axs = plt.subplots(nrows=4,ncols=2,figsize=(12, 24), sharex=False)
fig.subplots_adjust(hspace=.4,wspace=0.3)
# AM stim (10 Hz AM noise)
AM = 1.0
clustInfoMask = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                             (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &
                             (clustInfoAll['Region']!='TeA') &     
                            (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
total_n = len(clustInfoMask.index)
mouse_N = len(clustInfoMask.mouse.unique())
print(str(mouse_N)+ ' mice')
print(clustInfoMask.mouse.unique())
print(str(total_n)+ ' total units')
clustInfoENH = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']>=0))]
clustInfoSUPP = clustInfoMask.loc[((clustInfoMask['OdorEffect_v_'+str(AM)+'_NoLED']<0))]
Enh_n = len(clustInfoENH.index)
print(str(Enh_n)+' Enh. units')
Supp_n = len(clustInfoSUPP.index)
print(str(Supp_n)+' Supp. units')
#################################################################################################################
#### PLOT 1 : OMI SCATTER PLOT - 5E LEFT
plt.subplot(4,2,1)
OdoreffectEnh = clustInfoENH['OdorEffect_v_'+str(AM)+'_NoLED']
OdoreffectLEDEnh = clustInfoENH['OdorEffect_v_'+str(AM)+'_LED']
x = OdoreffectEnh
y = OdoreffectLEDEnh
OdoreffectSupp = clustInfoSUPP['OdorEffect_v_'+str(AM)+'_NoLED']
OdoreffectLEDSupp = clustInfoSUPP['OdorEffect_v_'+str(AM)+'_LED']
x2 = OdoreffectSupp
y2 = OdoreffectLEDSupp
print('SCATTER PLOT STATS:')
t, p = stats.shapiro(x)
print('Enhanced off shapiro p ='+str(p))
t, pp = stats.shapiro(y)
print('Enhanced ON shapiro p ='+str(pp))
if p<0.05 or pp<0.05:
    t, ep = stats.wilcoxon(x, y)
    print('Enhanced Wilcoxon W= '+str(t))
    print('Enhanced Wilcoxon p= '+str(ep))
else:
    t, ep = stats.ttest_rel(x, y)
    print('Enhanced Paired ttest t= '+str(t))
    print('Enhanced Paired ttest p= '+str(ep))
t, p = stats.shapiro(x2)
print('Suppressed off shapiro p ='+str(p))
t, pp = stats.shapiro(y2)
print('Suppressed ON shapiro p ='+str(pp))
if p<0.05 or pp<0.05:
    t, sp = stats.wilcoxon(x2, y2)
    print('Suppressed Wilcoxon W= '+str(t))
    print('Suppressed Wilcoxon p= '+str(sp))
else:
    t, sp = stats.ttest_rel(x2, y2)
    print('Suppressed Paired ttest t= '+str(t))
    print('Suppressed Paired ttest p= '+str(sp))
plt.scatter(x, y, color='mediumvioletred', s=20)
plt.scatter(x2, y2, color='violet', s=20)
plt.legend(['Enhanced, p<0.01', 'Suppressed, n.s.'], loc='upper left', prop={'size': 20})
xmin = -0.5
xmax = 0.5
ymin = -0.5
ymax = 0.5
plt.xlabel('OMI (LED off)', color='magenta')
plt.ylabel('OMI (LED ON)', color='deepskyblue')
plt.xlim([xmin, xmax])
plt.xticks([xmin, 0, xmax])
plt.ylim([ymin, ymax])
plt.yticks([ymin, 0, ymax])
plt.axhline(0, color='black', ls='-')
plt.axvline(0, color='black', ls='-')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
unity = np.linspace(np.min([xmin, ymin]), np.max([xmax, ymax]))
plt.plot(unity, unity, color='black', ls='--', label='_nolegend_')
# LED Modulation Index save pickle for Fig 6 comparison
AM_LEDeffectEnh_Expt = clustInfoENH['LEDindexOdor_'+str(AM)]
with open('Data_6Eright_LEDindexAM_stGtACR1.pickle', 'wb') as f:
    pickle.dump(AM_LEDeffectEnh_Expt, f)
#################################################################################################################
#### PLOT 2 : OMI BAR PLOTS - 5E RIGHT
plt.subplot(4,2,2)
data = {
        "LED off": [x.mean(), x2.mean()],
        "LED ON": [y.mean(), y2.mean()],
    }
errdata = {
        "LED off": [x.std()/np.sqrt(len(x)), x2.std()/np.sqrt(len(x2))],
        "LED ON": [y.std()/np.sqrt(len(y)), y2.std()/np.sqrt(len(y2))],
    }
bar_plot(axs[0,1], data, errdata, colors=['magenta', 'deepskyblue'], total_width=.8, single_width=.9, legend_loc='upper right')
plt.ylabel('OMI')
plt.xticks(ticks=[0, 1], labels=['Enhanced', 'Suppressed'])
plt.ylim([-0.2, 0.2])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 3 : ENHANCED DB LINE PLOT - 5F LEFT
plt.subplot(4,2,3)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
mean_dBarray_LED_NoOdor = []
mean_dBarray_LED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
sem_dBarray_LED_NoOdor = []
sem_dBarray_LED_Odor = []
print('ENHANCED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    LED_NoOdor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
    LED_Odor = np.array((clustInfoENH['zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoLED_NoOdor shapiro p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': NoLED_Odor shapiro. p= '+str(pp))
    t, ppp = stats.shapiro(LED_Odor)
    print(str(dB)+': LED_Odor shapiro p= '+str(ppp))
    if p<0.05 or pp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Wilcoxon W= '+str(t))
        print(str(dB)+' NoLED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Paired ttest t= '+str(t))
        print(str(dB)+' NoLED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    if p<0.05 or ppp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Wilcoxon W= '+str(t))
        print(str(dB)+' LED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Paired ttest t= '+str(t))
        print(str(dB)+' LED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    mean_dBarray_LED_NoOdor = np.append(mean_dBarray_LED_NoOdor, LED_NoOdor.mean())
    mean_dBarray_LED_Odor = np.append(mean_dBarray_LED_Odor, LED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
    sem_dBarray_LED_NoOdor = np.append(sem_dBarray_LED_NoOdor, LED_NoOdor.std()/np.sqrt(len(LED_NoOdor)))
    sem_dBarray_LED_Odor = np.append(sem_dBarray_LED_Odor, LED_Odor.std()/np.sqrt(len(LED_Odor)))
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_NoOdor[2:], sem_dBarray_NoLED_NoOdor[2:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_Odor[2:], sem_dBarray_NoLED_Odor[2:], capsize=5, marker='o', color='mediumvioletred')
plt.errorbar(sounds[2:]-50, mean_dBarray_LED_Odor[2:], sem_dBarray_LED_Odor[2:], capsize=5, marker='o', color='deepskyblue')
plt.legend(['Sound (control)', 'Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper left', prop={'size': 20})
plt.ylim([0, 30])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(np.asarray(sounds[2:]-50))
plt.ylabel('Avg. Firing Rate Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Enhanced')
#################################################################################################################
#### PLOT 4 : SUPPRESSED DB LINE PLOT - 5F RIGHT
plt.subplot(4,2,4)
mean_dBarray_NoLED_NoOdor = []
mean_dBarray_NoLED_Odor = []
mean_dBarray_LED_NoOdor = []
mean_dBarray_LED_Odor = []
sem_dBarray_NoLED_NoOdor = []
sem_dBarray_NoLED_Odor = []
sem_dBarray_LED_NoOdor = []
sem_dBarray_LED_Odor = []
print('SUPPRESSED STATS LINE PLOT:')
for i, dB in enumerate(np.asarray(sounds)):
    NoLED_NoOdor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_NoOdor']))
    NoLED_Odor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_NoLED_Odor']))
    LED_NoOdor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_LED_NoOdor']))
    LED_Odor = np.array((clustInfoSUPP['zResp_'+str(dB)+'_'+str(AM)+'_LED_Odor']))
    t, p = stats.shapiro(NoLED_NoOdor)
    print(str(dB)+': NoLED_NoOdor shapiro p= '+str(p))
    t, pp = stats.shapiro(NoLED_Odor)
    print(str(dB)+': NoLED_Odor shapiro. p= '+str(pp))
    t, ppp = stats.shapiro(LED_Odor)
    print(str(dB)+': LED_Odor shapiro p= '+str(ppp))
    if p<0.05 or pp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Wilcoxon W= '+str(t))
        print(str(dB)+' NoLED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, NoLED_Odor)
        print(str(dB)+' NoLED Paired ttest t= '+str(t))
        print(str(dB)+' NoLED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    if p<0.05 or ppp<0.05:
        t, ep = stats.wilcoxon(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Wilcoxon W= '+str(t))
        print(str(dB)+' LED Wilcoxon p= '+str(ep)+ ' x2= '+str(ep*2))
    else:
        t, ep = stats.ttest_rel(NoLED_NoOdor, LED_Odor)
        print(str(dB)+' LED Paired ttest t= '+str(t))
        print(str(dB)+' LED Paired ttest p= '+str(ep)+ ' x2= '+str(ep*2))
    mean_dBarray_NoLED_NoOdor = np.append(mean_dBarray_NoLED_NoOdor, NoLED_NoOdor.mean())
    mean_dBarray_NoLED_Odor = np.append(mean_dBarray_NoLED_Odor, NoLED_Odor.mean())
    mean_dBarray_LED_NoOdor = np.append(mean_dBarray_LED_NoOdor, LED_NoOdor.mean())
    mean_dBarray_LED_Odor = np.append(mean_dBarray_LED_Odor, LED_Odor.mean())
    sem_dBarray_NoLED_NoOdor = np.append(sem_dBarray_NoLED_NoOdor, NoLED_NoOdor.std()/np.sqrt(len(NoLED_NoOdor)))
    sem_dBarray_NoLED_Odor = np.append(sem_dBarray_NoLED_Odor, NoLED_Odor.std()/np.sqrt(len(NoLED_Odor)))
    sem_dBarray_LED_NoOdor = np.append(sem_dBarray_LED_NoOdor, LED_NoOdor.std()/np.sqrt(len(LED_NoOdor)))
    sem_dBarray_LED_Odor = np.append(sem_dBarray_LED_Odor, LED_Odor.std()/np.sqrt(len(LED_Odor)))
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_NoOdor[2:], sem_dBarray_NoLED_NoOdor[2:], capsize=5, marker='o', color='black')
plt.errorbar(sounds[2:]-50, mean_dBarray_NoLED_Odor[2:], sem_dBarray_NoLED_Odor[2:], capsize=5, marker='o', color='violet')
plt.errorbar(sounds[2:]-50, mean_dBarray_LED_Odor[2:], sem_dBarray_LED_Odor[2:], capsize=5, marker='o', color='lightskyblue')
plt.legend(['Sound (control)', 'Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper left', prop={'size': 20})
plt.ylim([0, 30])
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(np.asarray(sounds[2:]-50))
# plt.ylabel('Avg. Firing Rate Z-scored')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.title('Suppressed')
#################################################################################################################
#### PLOT 5 : ENHANCED STATS COMPARISON BAR - 5G LEFT
plt.subplot(4,2,5)
mean_OMIarray_NoLED = []
mean_OMIarray_LED = []
sem_OMIarray_NoLED = []
sem_OMIarray_LED = []
K_data_off = {}
K_data_ON = {}
print('ENHANCED STATS OMI BAR:')
for i, dB in enumerate(np.asarray(sounds[2:])):
    NoLED = np.array((clustInfoENH['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    LED = np.array((clustInfoENH['OMI_'+str(dB)+'_'+str(AM)+'_LED']))
    K_data_off[str(dB)] = NoLED[~np.isnan(NoLED)]
    K_data_ON[str(dB)] = LED[~np.isnan(LED)]
    t, p = stats.shapiro(NoLED)
    print(str(dB)+': LED off shap. p= '+str(p))
    t, pp = stats.shapiro(LED)
    print(str(dB)+': LED ON shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED, LED)
        print(str(dB)+' Wilcoxon W= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_rel(NoLED, LED)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, NoLED.mean())
    mean_OMIarray_LED = np.append(mean_OMIarray_LED, LED.mean())
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, NoLED.std()/np.sqrt(len(NoLED)))
    sem_OMIarray_LED = np.append(sem_OMIarray_LED, LED.std()/np.sqrt(len(LED)))
data = {
        "LED off": mean_OMIarray_NoLED,
        "LED ON": mean_OMIarray_LED,
    }
errdata = {
        "LED off": sem_OMIarray_NoLED,
        "LED ON": sem_OMIarray_LED,
    }
bar_plot(axs[2,0], data, errdata, colors=['mediumvioletred', 'deepskyblue'], total_width=.8, single_width=.9, legend_loc='lower left')
plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['-5', '5', '15', '25'])
plt.ylim([-0.2, 0.2])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 6 : SUPPRESSED STATS COMPARISON BAR - 5G RIGHT
plt.subplot(4,2,6)
mean_OMIarray_NoLED = []
mean_OMIarray_LED = []
sem_OMIarray_NoLED = []
sem_OMIarray_LED = []
K_data_off = {}
K_data_ON = {}
print('SUPPRESSED STATS OMI BAR:')
for i, dB in enumerate(np.asarray(sounds[2:])):
    NoLED = np.array((clustInfoSUPP['OMI_'+str(dB)+'_'+str(AM)+'_NoLED']))
    LED = np.array((clustInfoSUPP['OMI_'+str(dB)+'_'+str(AM)+'_LED']))
    K_data_off[str(dB)] = NoLED[~np.isnan(NoLED)]
    K_data_ON[str(dB)] = LED[~np.isnan(LED)]
    t, p = stats.shapiro(NoLED)
    print(str(dB)+': LED off shap. p= '+str(p))
    t, pp = stats.shapiro(LED)
    print(str(dB)+': LED ON shap. p= '+str(pp))
    if p<0.05 or pp<0.05:
        t, p = stats.wilcoxon(NoLED, LED)
        print(str(dB)+' Wilcoxon W= '+str(t))
        print(str(dB)+' Wilcoxon p= '+str(p))
    else:
        t, p = stats.ttest_rel(NoLED, LED)
        print(str(dB)+' Paired ttest t= '+str(t))
        print(str(dB)+' Paired ttest p= '+str(p))
    mean_OMIarray_NoLED = np.append(mean_OMIarray_NoLED, NoLED.mean())
    mean_OMIarray_LED = np.append(mean_OMIarray_LED, LED.mean())
    sem_OMIarray_NoLED = np.append(sem_OMIarray_NoLED, NoLED.std()/np.sqrt(len(NoLED)))
    sem_OMIarray_LED = np.append(sem_OMIarray_LED, LED.std()/np.sqrt(len(LED)))
data = {
        "LED off": mean_OMIarray_NoLED,
        "LED ON": mean_OMIarray_LED,
    }
errdata = {
        "LED off": sem_OMIarray_NoLED,
        "LED ON": sem_OMIarray_LED,
    }
bar_plot(axs[2,1], data, errdata, colors=['violet', 'lightskyblue'], total_width=.8, single_width=.9, legend_loc='upper right')
# plt.ylabel('OMI')
plt.xlabel('Sound Level (dB SNR)')
plt.xticks(ticks=[0, 1, 2, 3], labels=['-5', '5', '15', '25'])
plt.ylim([-0.2, 0.2])
plt.yticks([-0.2, 0, 0.2])
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 7 : ENHANCED PSTH - 5H LEFT
plt.subplot(4,2,7)
dB_sel = 75
clustInfoENH = clustInfoMask.loc[(clustInfoMask['OMI_'+str(dB_sel)+'_'+str(AM)+'_NoLED']>0)]
n = len(clustInfoENH.index)
PSTHall_LED_NoOdor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_NoOdor'])))
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_LED_Odor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_Odor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoENH['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanLED_NoOdor = PSTHall_LED_NoOdor.mean(axis=0)
semLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanLED_Odor = PSTHall_LED_Odor.mean(axis=0)
semLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='mediumvioletred')
plt.plot(time, smooth(meanLED_Odor, 3), color='deepskyblue')
plt.legend(['Sound (control)','Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper right', prop={'size': 20})
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate Z-Scored')
plt.ylim(-5, 60)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='-')
plt.axvline(x=0, color='deepskyblue', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')
#################################################################################################################
#### PLOT 8 : SUPPRESSED PSTH - 5H RIGHT
plt.subplot(4,2,8)
dB_sel = 75
clustInfoSUPP = clustInfoMask.loc[(clustInfoMask['OMI_'+str(dB_sel)+'_'+str(AM)+'_NoLED']<0)]
n = len(clustInfoSUPP.index)
PSTHall_LED_NoOdor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_NoOdor'])))
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_LED_Odor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_Odor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoSUPP['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanLED_NoOdor = PSTHall_LED_NoOdor.mean(axis=0)
semLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanLED_Odor = PSTHall_LED_Odor.mean(axis=0)
semLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='violet')
plt.plot(time, smooth(meanLED_Odor, 3), color='lightskyblue')
plt.legend(['Sound (control)','Odor+Sound (LED off)', 'Odor+Sound (LED ON)'], loc='upper right', prop={'size': 20})
plt.xlabel('Time (s)')
# plt.ylabel('Avg. Firing Rate Z-Scored')
plt.ylim(-5, 60)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title(str(dB_sel-50)+ ' dB SNR')
plt.axvline(x=0, color='magenta', ls='-')
plt.axvline(x=0, color='deepskyblue', ls='--')
plt.axvline(x=0.250, color='gray', ls='--')
plt.axvline(x=1.250, color='gray', ls='--')
plt.axhline(y=0, color='black', ls='--')

# Save Fig
filename = 'Fig 5EFGH'+'.pdf'
plt.savefig(filename, bbox_inches='tight', dpi=300, transparent=False)

plt.show()

In [ ]:
# FIGURE 5 I-J

fig, axs = plt.subplots(nrows=1,ncols=5,figsize=(30, 5), sharex=False)
fig.subplots_adjust(hspace=.4,wspace=0.3)
AM = 1.0
clustInfoMask = clustInfoAll.loc[((clustInfoAll['GLM_AM_LEDoff_dB_p']<0.05) &
                             (clustInfoAll['GLM_AM_LEDoff_dB_coef']>0) &
                             (clustInfoAll['Region']!='TeA') &     
                            (clustInfoAll['BL_FRmean']>1) &
                                ((clustInfoAll['GLM_AM_LEDoff_Odor_p']<0.05) |
                                (clustInfoAll['GLM_AM_LEDoff_Interaction_p']<0.05)))]
print('n= '+str(len(clustInfoMask.index)))
Resp_0_NoOdorNoLED = np.array(clustInfoMask['zResp_'+str(0)+'_'+str(AM)+'_NoLED_NoOdor'])
Resp_0_OdorNoLED = np.array(clustInfoMask['zResp_'+str(0)+'_'+str(AM)+'_NoLED_Odor'])                        
Resp_0_NoOdorLED = np.array(clustInfoMask['zResp_'+str(0)+'_'+str(AM)+'_LED_NoOdor'])
Resp_0_OdorLED = np.array(clustInfoMask['zResp_'+str(0)+'_'+str(AM)+'_LED_Odor'])  
############################################################################
#### 0 DB : I
plt.subplot(1,5,1)
x = Resp_0_NoOdorNoLED
y = Resp_0_OdorNoLED
x2 = Resp_0_NoOdorLED
y2 = Resp_0_OdorLED
axmin=-10
axmax=10
plt.scatter(x, y, color='black', s=20)
plt.scatter(x, y2, color='deepskyblue', s=20)
plt.xlabel('Control')
plt.ylabel('Odor')
plt.title('0 dB')
plt.xlim([axmin, axmax])
plt.ylim([axmin, axmax])
plt.yticks([-5, 0, 5])
plt.xticks([-5, 0, 5])
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
unity = np.linspace(axmin, axmax)
plt.plot(unity, unity, color='black', ls='--', label='_nolegend_')
print('# 0 DB STATS: #')
print('LED off STATS:')
t, p = stats.shapiro(x)
print('control shapiro= '+str(p))
t, p = stats.shapiro(y)
print('Odor shapiro= '+str(p))
t, op = stats.ttest_rel(x, y)
print('LED off ttest t = '+str(t))
print('LED off ttest p = '+str(op))
print('LED ON STATS:')
t, p = stats.shapiro(x)
print('control shapiro= '+str(p))
t, p = stats.shapiro(y2)
print('Odor shapiro= '+str(p))
t, lp = stats.wilcoxon(x, y2)
print('LED ON wilcoxon W = '+str(t))
print('LED ON wilcoxon p = '+str(lp))
plt.legend(['LED off, p<0.001', 'LED ON, n.s.'], loc='lower left', prop={'size': 20})
############################################################################
plt.subplot(1,5,2)
OMI_0_NoLED = np.array(clustInfoMask['OMI_'+str(0)+'_'+str(AM)+'_NoLED'])
OMI_0_LED = np.array(clustInfoMask['OMI_'+str(0)+'_'+str(AM)+'_LED'])
x = OMI_0_NoLED
y = OMI_0_LED
bothmeans = [x.mean(), y.mean()]
botherr = [x.std()/np.sqrt(len(x)), y.std()/np.sqrt(len(y))]
plt.bar(['LED off', 'LED ON'], bothmeans, color=['magenta', 'deepskyblue'], edgecolor=['black', 'black'], yerr=botherr, capsize=10, fill=True)
plt.tick_params(bottom = False)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.axhline(y=0, color='black', ls='--')
plt.ylabel('OMI')
plt.ylim([-0.1, 0.3])
plt.yticks([-0.1, 0, 0.1, 0.2, 0.3])
plt.title('0 dB')
print('OMI BAR STATS:')
t,p = stats.shapiro(x)
print('LED off shapiro p = '+ str(p))
t,p = stats.shapiro(y)
print('LED ON shapiro p = '+ str(p))
t, p = stats.wilcoxon(x, y)
print('OMI wilcoxon W = '+str(t))
print('OMI wilcoxon p = '+str(p))
############################################################################
plt.subplot(1,5,3)
dB_sel = 0
PSTHall_LED_NoOdor = np.vstack(np.array((clustInfoMask['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_NoOdor'])))
PSTHall_NoLED_NoOdor = np.vstack(np.array((clustInfoMask['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_NoOdor'])))
PSTHall_LED_Odor = np.vstack(np.array((clustInfoMask['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_LED_Odor'])))
PSTHall_NoLED_Odor = np.vstack(np.array((clustInfoMask['zPSTH_'+str(dB_sel)+'_'+str(AM)+'_NoLED_Odor'])))
meanLED_NoOdor = PSTHall_LED_NoOdor.mean(axis=0)
semLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanNoLED_NoOdor = PSTHall_NoLED_NoOdor.mean(axis=0)
semNoLED_NoOdor = PSTHall_LED_NoOdor.std(axis=0)/np.sqrt(n)
meanLED_Odor = PSTHall_LED_Odor.mean(axis=0)
semLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
meanNoLED_Odor = PSTHall_NoLED_Odor.mean(axis=0)
semNoLED_Odor = PSTHall_LED_Odor.std(axis=0)/np.sqrt(n)
plt.plot(time, smooth(meanNoLED_NoOdor, 3), color='black')
plt.plot(time, smooth(meanNoLED_Odor, 3), color='magenta')
plt.plot(time, smooth(meanLED_Odor, 3), color='deepskyblue')
plt.legend(['Control','Odor (LED off)', 'Odor (LED ON)'], loc='upper right', prop={'size': 20})
plt.xlabel('Time (s)')
plt.ylabel('Avg. Firing Rate Z-Scored')
plt.ylim(-2, 8)
plt.xlim(-0.25, 1.5)
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.margins(x=0)
plt.xticks([0, 0.5, 1, 1.5])
plt.title('0 dB')
plt.axvline(x=0, color='magenta', ls='-')
plt.axvline(x=0, color='deepskyblue', ls='--')
plt.axhline(y=0, color='black', ls='--')
############################################################################
#### ANALYSIS : SNIFF FREQUENCY AND AMPLITUDE
sessionlist = np.unique(clustInfoAll['blockname'])
Sess_Odor_freq = np.empty(len(sessionlist))
Sess_NoOdor_freq = np.empty(len(sessionlist))
Sess_OdorLED_freq = np.empty(len(sessionlist))
Sess_NoOdorLED_freq = np.empty(len(sessionlist))
Sess_Odor_amp = np.empty(len(sessionlist))
Sess_NoOdor_amp = np.empty(len(sessionlist))
Sess_OdorLED_amp = np.empty(len(sessionlist))
Sess_NoOdorLED_amp = np.empty(len(sessionlist))
for b, blockname in enumerate(sessionlist):
    ind = clustInfoAll.index[clustInfoAll['blockname']==blockname].tolist()[0]
    indVial = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'Vial_stim'])
    indLED = np.array(clustInfoAll.at[clustInfoAll.index[ind], 'LED_stim'])
    OdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'OdorVial']
    NoOdorVial = clustInfoAll.at[clustInfoAll.index[ind], 'NoOdorVial']
    sniff_timevect = clustInfoAll.at[clustInfoAll.index[ind], 'sniff_timevect'][0]
    Sniffs = clustInfoAll.at[clustInfoAll.index[ind], 'Sniffs'][0]
    time_mask = (sniff_timevect>0) & (sniff_timevect<1.25)
    Sniffs = Sniffs[:,time_mask]
    sniff_timevect = sniff_timevect[time_mask]
    fs = 30000
    freq_cutoff = 12
    distance = int((1/freq_cutoff)*fs)
    Alltrialfreq = np.empty(len(indVial))
    Alltrialamp = np.empty(len(indVial))
    for t in range(len(indVial)):
        peaks,_ = find_peaks(Sniffs[t,:], distance=distance, prominence=(30, None))
        peaks2,_ = find_peaks(-Sniffs[t,:], distance=distance, prominence=(30, None))
        peaks_diffs = np.append(np.diff(peaks), np.diff(peaks2))
        avg_peakdiff = peaks_diffs.mean()
        Freq = fs / avg_peakdiff
        Alltrialfreq[t] = Freq
        peak1 = np.nanmean(Sniffs[t,:][peaks])
        peak2 = np.nanmean(Sniffs[t,:][peaks2])
        avg_peakamp = (peak1 - peak2) / 10
        Alltrialamp[t] = avg_peakamp
    Odor_freq = Alltrialfreq[((indVial==OdorVial) & (indLED==0))]
    NoOdor_freq = Alltrialfreq[((indVial==NoOdorVial) & (indLED==0))]
    OdorLED_freq = Alltrialfreq[((indVial==OdorVial) & (indLED==1))]
    NoOdorLED_freq = Alltrialfreq[((indVial==NoOdorVial) & (indLED==1))]
    Odor_amp = Alltrialamp[((indVial==OdorVial) & (indLED==0))]
    NoOdor_amp = Alltrialamp[((indVial==NoOdorVial) & (indLED==0))]
    OdorLED_amp = Alltrialamp[((indVial==OdorVial) & (indLED==1))]
    NoOdorLED_amp = Alltrialamp[((indVial==NoOdorVial) & (indLED==1))]
    Avg_Odor_freq = np.nanmean(Odor_freq)
    Avg_Odor_amp = np.nanmean(Odor_amp)
    Avg_NoOdor_freq = np.nanmean(NoOdor_freq)
    Avg_NoOdor_amp = np.nanmean(NoOdor_amp)
    Avg_OdorLED_freq = np.nanmean(OdorLED_freq)
    Avg_OdorLED_amp = np.nanmean(OdorLED_amp)
    Avg_NoOdorLED_freq = np.nanmean(NoOdorLED_freq)
    Avg_NoOdorLED_amp = np.nanmean(NoOdorLED_amp)
    Sess_Odor_freq[b] = Avg_Odor_freq
    Sess_NoOdor_freq[b] = Avg_NoOdor_freq
    Sess_OdorLED_freq[b] = Avg_OdorLED_freq
    Sess_NoOdorLED_freq[b] = Avg_NoOdorLED_freq
    Sess_Odor_amp[b] = Avg_Odor_amp
    Sess_NoOdor_amp[b] = Avg_NoOdor_amp
    Sess_OdorLED_amp[b] = Avg_OdorLED_amp
    Sess_NoOdorLED_amp[b] = Avg_NoOdorLED_amp
############################################################################
#### SNIFF : J
plt.subplot(1,5,4)
plt.scatter(Sess_NoOdor_freq, Sess_Odor_freq, color='black', s=20)
plt.scatter(Sess_NoOdorLED_freq, Sess_OdorLED_freq, color='deepskyblue', s=20)
plt.title('Sniff Frequency (Hz)')
plt.xlabel('Control')
plt.ylabel('Odor')
axmin = 2.5
axmax = 6.5
plt.xlim([axmin, axmax])
plt.ylim([axmin, axmax])
plt.yticks([3, 4, 5, 6])
plt.xticks([3, 4, 5, 6])
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
unity = np.linspace(axmin, axmax)
plt.plot(unity, unity, color='black', ls='--', label='_nolegend_')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.legend(['LED off, n.s.', 'LED ON, n.s.'], loc='upper left', prop={'size': 20})
print('# FREQUENCY STATS: #')
print('n = '+str(len(sessionlist))+' sessions')
print('LED off STATS:')
t,p = stats.shapiro(Sess_NoOdor_freq)
print('control shapiro p = '+ str(p))
t,p = stats.shapiro(Sess_Odor_freq)
print('Odor shapiro p = '+ str(p))
t,p = stats.ttest_rel(Sess_NoOdor_freq, Sess_Odor_freq)
print('LED off ttest t = '+str(t))
print('LED off ttest p = '+str(p))
print('LED ON STATS:')
t,p = stats.shapiro(Sess_NoOdorLED_freq)
print('control shapiro p = '+ str(p))
t,p = stats.shapiro(Sess_OdorLED_freq)
print('Odor shapiro p = '+ str(p))
t,p = stats.ttest_rel(Sess_NoOdorLED_freq, Sess_OdorLED_freq)
print('LED ON ttest t = '+str(t))
print('LED ON ttest p = '+str(p))
print('ODOR VS ODOR LED:')
t,p = stats.ttest_rel(Sess_Odor_freq, Sess_OdorLED_freq)
print('LED ON ttest t = '+str(t))
print('LED ON ttest p = '+str(p))
############################################################################
plt.subplot(1,5,5)
plt.scatter(Sess_NoOdor_amp, Sess_Odor_amp, color='black', s=20)
plt.scatter(Sess_NoOdorLED_amp, Sess_OdorLED_amp, color='deepskyblue', s=20)
plt.title('Sniff Amplitude (a.u.)')
plt.xlabel('Control')
plt.ylabel('Odor')
axmin = 0
axmax = 200
plt.xlim([axmin, axmax])
plt.ylim([axmin, axmax])
plt.yticks([50, 100, 150])
plt.xticks([50, 100, 150])
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
unity = np.linspace(axmin, axmax)
plt.plot(unity, unity, color='black', ls='--', label='_nolegend_')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
print('# AMPLITUDE STATS: # ')
print('n = '+str(len(sessionlist))+' sessions')
print('LED off STATS:')
t,p = stats.shapiro(Sess_NoOdor_amp)
print('control shapiro p = '+ str(p))
t,p = stats.shapiro(Sess_Odor_amp)
print('Odor shapiro p = '+ str(p))
t,p = stats.wilcoxon(Sess_NoOdor_amp, Sess_Odor_amp)
print('LED off wilcoxon W = '+str(t))
print('LED off wilcoxon p = '+str(p))
print('LED ON STATS:')
t,p = stats.shapiro(Sess_NoOdorLED_amp)
print('control shapiro p = '+ str(p))
t,p = stats.shapiro(Sess_OdorLED_amp)
print('Odor shapiro p = '+ str(p))
t,p = stats.wilcoxon(Sess_NoOdorLED_amp, Sess_OdorLED_amp)
print('LED ON wilcoxon W = '+str(t))
print('LED ON wilcoxon p = '+str(p))
print('ODOR VS ODOR LED:')
t,p = stats.wilcoxon(Sess_Odor_amp, Sess_OdorLED_amp)
print('LED ON wilcoxon W = '+str(t))
print('LED ON wilcoxon p = '+str(p))
plt.legend(['LED off, p<0.001', 'LED ON, p<0.001'], loc='upper left', prop={'size': 20})

# Save Fig
filename = 'Fig 5IJ'+'.pdf'
plt.savefig(filename, bbox_inches='tight', dpi=300, transparent=False)

plt.show()